<a href="https://colab.research.google.com/github/chengyupku/On-the-steerability-of-generative-adversarial-networks/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tensorflow

In [0]:
!pip install tensorlayer


In [0]:
lr=0.0001
num_samples=200

In [0]:
import os
import io
import IPython.display
import numpy as np
import PIL.Image
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import time
import tensorlayer as tl
import time

In [5]:
tf.__version__

'2.1.0'

In [0]:
hub_url='https://tfhub.dev/deepmind/biggan-256/2'
hub_layer=hub.KerasLayer(hub_url)

In [0]:
model=tf.keras.Sequential()
model.add(hub_layer)
trunc=1.0
batch_start=0
batch_size=8
Nsliders=1
w = tf.Variable(np.random.normal(0.0, 0.1, [1, 140, Nsliders]),
        name='walk', dtype=np.float32)
img_size=256
num_channels=3
optimizer=tf.keras.optimizers.Adam(lr)

In [0]:
def get_target_np(outputs_zs, alpha):
    
    mask_fn = np.ones(outputs_zs.shape)
    
    if alpha == 1:
        mask_fn=tf.convert_to_tensor(mask_fn,dtype=tf.float32)
        return outputs_zs, mask_fn
    
    new_size = int(alpha*img_size)
    outputs_zs_array=outputs_zs.numpy()
    ## crop
    if alpha < 1:
        output_cropped = outputs_zs_array[:,img_size//2-new_size//2:img_size//2+new_size//2, img_size//2-new_size//2:img_size//2+new_size//2,:]
        mask_cropped = mask_fn
    
    ## padding
    else:
        output_cropped = np.zeros((outputs_zs.shape[0], new_size, new_size, outputs_zs.shape[3]))
        mask_cropped = np.zeros((outputs_zs.shape[0], new_size, new_size, outputs_zs.shape[3]))
        output_cropped[:, new_size//2-img_size//2:new_size//2+img_size//2, new_size//2-img_size//2:new_size//2+img_size//2,:] = outputs_zs_array
        mask_cropped[:, new_size//2-img_size//2:new_size//2+img_size//2, new_size//2-img_size//2:new_size//2+img_size//2,:] = mask_fn
    
    ## Resize
    target_fn = np.zeros(outputs_zs.shape)
    mask_out = np.zeros(outputs_zs.shape)
    for i in range(outputs_zs.shape[0]):
        target_fn[i,:,:,:] = cv2.resize(output_cropped[i,:,:,:], (img_size, img_size), interpolation = cv2.INTER_LINEAR)
        mask_out[i,:,:,:] = cv2.resize(mask_cropped[i,:,:,:], (img_size, img_size), interpolation = cv2.INTER_LINEAR)
        
    mask_out[np.nonzero(mask_out)] = 1.
    assert(np.setdiff1d(mask_out, [0., 1.]).size == 0)
    target_fn=tf.convert_to_tensor(target_fn,dtype=tf.float32)
    mask_out=tf.convert_to_tensor(mask_out,dtype=tf.float32)
    return target_fn, mask_out

In [0]:
def train():
  trunc=1.0
  zs=trunc*tf.random.truncated_normal([num_samples,140])
  ys=tf.random.uniform([num_samples],maxval=1000, dtype=tf.int32)
  ys=tf.one_hot(ys,1000)
  Loss_sum=0
  n_epoch=1
  Loss_sum_iter=0
  optim_iter=0
  batch_size=8
  for epoch in range(n_epoch):
    for batch_start in range(0,num_samples,batch_size):
      coin=np.random.uniform(0,1)
      if coin <=0.5:
        alpha_val=np.random.uniform(0.25,1.)
      else:
        alpha_val=np.random.uniform(1.,4.)
      s=slice(batch_start,min(num_samples,batch_start+batch_size))
      out_zs=model(dict(y=ys[s],z=zs[s],truncation=trunc))
      out_zs=tf.cast(out_zs,tf.float32)
      print('go on')
      target_fn,mask_out=get_target_np(out_zs,alpha_val)
      alpha_val_for_graph = np.ones((zs[s].shape[0], Nsliders)) * np.log(alpha_val)
      z1=zs[s]
      with tf.GradientTape() as tape:
        for i in range(Nsliders):
          tmp=tf.expand_dims(alpha_val_for_graph[:,i],axis=1)
          tmp=tf.cast(tmp,tf.float32)
          z1=z1+tmp*w[:,:,i] 
          transout_zs=model(dict(y=ys[s],z=z1,truncation=trunc))
          transout_zs=tf.cast(transout_zs,tf.float32)
          loss=tf.compat.v1.losses.compute_weighted_loss(tf.square(transout_zs-target_fn),weights=mask_out)
      grads=tape.gradient(loss,[w])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[w]))

In [0]:
train()